In [1]:
!/opt/venv/bin/python -m pip install --upgrade pip
!pip install tweepy==3.10.0

In [93]:
import re
import ast
import pandas as pd
import demoji
import string
import nltk
import tweepy
import webbrowser
nltk.download('wordnet') 
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk import sent_tokenize, word_tokenize, pos_tag
from emoji.unicode_codes import UNICODE_EMOJI
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [92]:
demoji.download_codes()

... OK (Got response in 0.17 seconds)
Writing emoji data to /home/jovyan/.demoji/codes.json ...
... OK


In [11]:
FIRST_PERSON_PRONOUNS = {
    'i', 'me', 'my', 'mine', 'we', 'us', 'our', 'ours'}
SECOND_PERSON_PRONOUNS = {
    'you', 'your', 'yours', 'u', 'ur', 'urs'}
THIRD_PERSON_PRONOUNS = {
    'he', 'him', 'his', 'she', 'her', 'hers', 'it', 'its', 'they', 'them',
    'their', 'theirs'}

SLANG = {
    'smh', 'fwb', 'lmfao', 'lmao', 'lms', 'tbh', 'rofl', 'wtf', 'bff',
    'wyd', 'lylc', 'brb', 'atm', 'imao', 'sml', 'btw', 'bw', 'imho', 'fyi',
    'ppl', 'sob', 'ttyl', 'imo', 'ltr', 'thx', 'kk', 'omg', 'omfg', 'ttys',
    'afn', 'bbs', 'cya', 'ez', 'f2f', 'gtr', 'ic', 'jk', 'k', 'ly', 'ya',
    'nm', 'np', 'plz', 'ru', 'so', 'tc', 'tmi', 'ym', 'ur', 'u', 'sol', 'fml'}

PUNCTUATION_LIST = list(string.punctuation)

## Data Collection

#### 1. Connect to Tweeter API

In [5]:
search_words = "Covid19 vaccine -filter:retweets"
date_since = "2020-11-01"

In [6]:
consumer_key = 'Jg7hPGYF2vRVJ8LsIsdaaC2CL'
consumer_secret = 'vgvuFr3IirJiKby3a2foMj92TyeRTYNZ4l4VcBg5R18fJLbRTD'

In [7]:
callback_uri = 'oob' # https://cfe.sh/twitter/callback

In [8]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret, callback_uri)
redirect_url = auth.get_authorization_url()
print(redirect_url)

https://api.twitter.com/oauth/authorize?oauth_token=OS_lpAAAAAABK43RAAABdt5BOqU


In [9]:
webbrowser.open(redirect_url)

False

In [10]:
user_pint_input = input("What's the pin?")

KeyboardInterrupt: Interrupted by user

In [ ]:
auth.get_access_token(user_pint_input)

('1341422445954252801-toTtbvlDQ6zp2n21Z5i9Aso3qKzVCd',
 'e3Ul5lu4cPcH3P3l0J2e4khoyjOTbRfdZQyC7xqdyJEoA')

In [ ]:
print(auth.access_token, auth.access_token_secret)

1341422445954252801-toTtbvlDQ6zp2n21Z5i9Aso3qKzVCd e3Ul5lu4cPcH3P3l0J2e4khoyjOTbRfdZQyC7xqdyJEoA


In [ ]:
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

#### 2. Choose potential features from JSON response

In [ ]:
# important features: created_at, hashtags, retweet_count, location
keys = [
    "created_at",
    "id",
    "full_text",
    "entities",
    "source",
    "user",
    "coordinates",
    "place",
    "is_quote_status",
    "retweet_count",
    "favorite_count",
    "possibly_sensitive",
]

#### 3. Scraping tweets

In [ ]:
def compose_dict_obj(raw_data, keys):
    """
    Return a dictionary of selected keys from raw_data
    """
    d = {}
    for key in keys:
        d[key] = raw_data.get(key)
    return d

In [ ]:
# the handler is time.sleep(15 * 60) if we reach the rate limit.
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.TweepError:
            time.sleep(15 * 60)

In [ ]:
csv_file = "data/raw/raw.csv"
tweetCursor = tweepy.Cursor(api.search, q=search_words, lang="en", tweet_mode="extended").items(30000)
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=keys)
        writer.writeheader()
        for i, tweet in enumerate(tweetCursor):
            if i % 1000 == 0:
                if i == 30000:
                    break
                print(i)
            big_json = tweet._json
            if 'retweeted_status' in big_json:
                data = big_json["retweeted_status"]
            else:
                data = big_json
            struct_data = compose_dict_obj(data, keys)
            writer.writerow(struct_data)$
except IOError:
    print("I/O error")

0
1000
2000


Rate limit reached. Sleeping for: 798


3000


#### 4. Combine datasets

In [ ]:
def merge_two_csvs(f1, f2):
    """
    Merge file1 to file2.
    """
    reader = csv.reader(open(f1))
    f = open(f2, "a+")
    writer = csv.writer(f)

    for row in reader:
        writer.writerow(row)
    f.close()

## Data Cleaning

#### Extract location from `user` and hashtags from `entites`

In [ ]:
class JsonParser:
    
    def __init__(self, json_string):
        self.json_string = json_string
        
    def convert_to_dict(self):
        '''Convert a json string to a Python dictionary object'''
        self.json_dict = ast.literal_eval(self.json_string)
        return self.json_dict

    def extract_hashtags(self):
        '''Extract hashtag contents in a json dictionary object, as a list'''
        self.convert_to_dict()
        hashtags_list = []
        for text in self.json_dict['hashtags']:
            hashtags_list.append(text['text'])
        
        return hashtags_list

    def extract_location(self):
        '''Extract location information in a json dictionary object, as a string'''
        self.convert_to_dict()
        return self.json_dict['location']

In [ ]:
json_str_entities = "{'hashtags': [{'text': 'COVID19', 'indices': [91, 99]}, {'text': 'EUvaccinationdays', 'indices': [224, 242]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/4Xa17PQkMv', 'expanded_url': 'https://www.pscp.tv/Ursulavonderleyen/1yoJMAOEyXlJQ', 'display_url': 'pscp.tv/Ursulavonderle…', 'indices': [245, 268]}]}"
json_str_user = "{'id': 124237063, 'id_str': '124237063', 'name': 'Francis S. Collins', 'screen_name': 'NIHDirector', 'location': 'Bethesda, Maryland, USA', 'description': 'Official Twitter account of Francis S. Collins, M.D., Ph.D., NIH Director.  NIH…Turning Discovery Into Health ®. Privacy Policy: https://t.co/QTI46zY00q', 'url': 'http://t.co/chM4AXez5s', 'entities': {'url': {'urls': [{'url': 'http://t.co/chM4AXez5s', 'expanded_url': 'http://www.nih.gov/about/director/', 'display_url': 'nih.gov/about/director/', 'indices': [0, 22]}]}, 'description': {'urls': [{'url': 'https://t.co/QTI46zY00q', 'expanded_url': 'http://bit.ly/2jQa5Rk', 'display_url': 'bit.ly/2jQa5Rk', 'indices': [129, 152]}]}}, 'protected': False, 'followers_count': 158635, 'friends_count': 129, 'listed_count': 2256, 'created_at': 'Thu Mar 18 18:32:15 +0000 2010', 'favourites_count': 114, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': True, 'statuses_count': 3475, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': '022330', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme15/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme15/bg.png', 'profile_background_tile': False, 'profile_image_url': 'http://pbs.twimg.com/profile_images/1282408119889530880/FMvG8yam_normal.jpg', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1282408119889530880/FMvG8yam_normal.jpg', 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/124237063/1354810851', 'profile_link_color': '0084B4', 'profile_sidebar_border_color': 'A8C7F7', 'profile_sidebar_fill_color': 'C0DFEC', 'profile_text_color': '333333', 'profile_use_background_image': True, 'has_extended_profile': False, 'default_profile': False, 'default_profile_image': False, 'following': False, 'follow_request_sent': False, 'notifications': False, 'translator_type': 'none'}"

entity = JsonParser(json_str_entities)
user = JsonParser(json_str_user)

assert isinstance(entity.convert_to_dict(), dict)
assert isinstance(user.convert_to_dict(), dict)
assert entity.extract_hashtags() == ['COVID19', 'EUvaccinationdays']
assert user.extract_location() == 'Bethesda, Maryland, USA'
print("success!")

success!


#### Remove duplicates

In [ ]:
def remove_duplicates(df, by=["full_text"]):
    """
    Remove duplicates from raw data file by specific columns and save results in file with name given.
    """
    boolean_mask = df.duplicated(subset=by, keep="first")
    df = df[~boolean_mask]
    return df

#### Split locations into the format of city, province and country

In [113]:
def split_city_province(text, mode=0):
    '''Split city and province in the location column'''
    if ", " in text and len(text.split(", ")) == 2 and mode == 0:
        city, province = text.split(', ')
        return city
    elif ", " in text and len(text.split(", ")) == 2 and mode == 1:
        city, province = text.split(', ')
        return province
    else:
        return text

#### Preprocessing `full_text` column

##### Exclude mentions and urls

In [ ]:
def process_tweets(text):
    '''Exclude mentions, urls, and html reference characters 
    in a string using regular expression'''
    text = re.sub('(\@|https:\/\/)\S+', '', text) #remove mentions and urls
    text = re.sub(r"&[a-z]+;", "", text) #exclude html reference characters
    return text

In [ ]:
text = "I'll be hosting on @CNN tomorrow from 9 to 11 AM - giving the latest on the Nashville explosien, Covid vaccine, and congressional battles over relief. Tune in! https://t.co/ZACOxL5q5S https://t.co/cDVAQIYrkR"

assert re.search (r"(\@|https:\/\/)\S+", text) != None
assert re.search (r"(\@|https:\/\/)\S+", process_tweets(text)) == None
print("success!")

#### Convert the `entities` column to number of hash tags

In [ ]:
# Remove duplicates
# df = remove_duplicates(df, by=["full_text"])

# Extract hashtags from entities column (raw data)
#df["entities"] = df["entities"].apply(lambda x: JsonParser(x).extract_hashtags())

# Extract location from user column (raw data)
#df["location"] = df["user"].apply(lambda x: JsonParser(x).extract_location())

# Split the location into city, province and country
# df["city"] = df["location"].apply(split_city_province)
# df["province"] = df["location"].apply(split_city_province, mode=1)
# df["country"] = ""


# create a column called `count_hashtags` that represnets the number of hastags
# df["count_hashtags"] =df["entitiles"].apply(lambda x: len(x))


## Feature Extraction

In [ ]:
def replace_null(text):
    if text == "":
        return text.replace("", "NaN")
    else:
        return text

In [ ]:
df["location"] = df["location"].apply(replace_null)

In [ ]:
df["hashtags_count"] = df["entities"].apply(count_hashtags)

### Feature Extract Functions

In [17]:
# imports

########################################################
# The verb dictionary used in this code is adapted from https://github.com/monolithpl/verb.forms.dictionary
########################################################

verb_dict = pd.read_csv('data/helper_dicts/verbs-dictionaries.csv', delimiter='\t', header=None, names=['present_simple', 'past_simple', 'past_participle', 'present_participle'])
past_simples = verb_dict['past_simple'].to_dict().values()

#### The number of past-tense verbs

In [115]:
# number of past-tense verbs
def count_past_tense(text):
    counter = 0
    tokens = word_tokenize(text.lower())
    tagged_words = pos_tag(tokens)
    
    for word, pos in tagged_words:
        if pos[0] == 'V':
            if word in past_simples:
                counter += 1
                
    return counter

In [122]:
test_no_past = 'I want to order a bunch of stuff from IKEA.'
test_past = 'I ordered a bunch of stuff from IKEA.'
test_past2 = "It began, and I was excited."

assert not count_past_tense(test_no_past)
assert count_past_tense(test_past) == 1
assert count_past_tense(test_past2) == 3

#### The number of first person pronouns

In [20]:
def count_first_person_pro(text):
    return len(
        re.findall(r"\b({})\b".format("|".join(FIRST_PERSON_PRONOUNS)), text.lower())
    )

In [120]:
assert count_first_person_pro("He likes his car and wants to give his friends for a drive.") == 0
assert count_first_person_pro("I am very proud of myself. My cat really likes me.") == 3
assert count_first_person_pro("You, he and I should go together so that none of us will be missing.") == 2

#### The nunber of second person pronouns

In [22]:
def count_second_person_pro(text):
    return len(
        re.findall(r"\b({})\b".format("|".join(SECOND_PERSON_PRONOUNS)), text.lower())
    )

In [121]:
assert count_second_person_pro("You, he and I should go together so that none of us will be missing.") == 1
assert count_second_person_pro("This is yours. You should always keep it with u.") == 3
assert count_second_person_pro("All of us are graduate students.") == 0

#### The number of third person pronouns

In [24]:
def count_third_person_pro(text):
    return len(
        re.findall(r"\b({})\b".format("|".join(THIRD_PERSON_PRONOUNS)), text.lower())
    )

In [25]:
assert count_third_person_pro("You, he and I should go together so that none of us will be missing.") == 1
assert count_third_person_pro("They are very kind. So don't be scared. He will tell you how to contact her.") == 3
assert count_third_person_pro("All of us are graduate students.") == 0

#### The number of coordinating conjunctions

In [26]:
def count_coord_conj(text):
    token_tag_pairs = pos_tag(word_tokenize(text.lower()))
    return len([p[1] for p in token_tag_pairs if p[1] == "CC"])

In [27]:
assert count_coord_conj("You, he and I should go together so that none of us will be missing.") == 1
assert count_coord_conj("But we could not deny that either he or she would be successful in the future.") == 3
assert count_coord_conj("All of us are graduate students.") == 0

#### The number of future-tense verbs

In [28]:
# number of future-tense verbs
def count_future_tense(text):
    future_form = {"'ll", "will", "wo"}
    counter = 0
    tokens = word_tokenize(text.lower())
    tagged_words = pos_tag(tokens)
    
    for word, pos in tagged_words:
        if pos == 'MD':
            if word in future_form:
                counter += 1
    return counter

In [29]:
test_future = "I'll buy you a cup of coffee."
test_future2 = "I will not go back to school."
test_future3 = "I won't be able to make it."
test_future4 = "I could do that."

assert count_future_tense(test_future) == 1
assert count_future_tense(test_future2) == 1
assert count_future_tense(test_future3) == 1
assert not count_future_tense(test_future4)

#### The number of commas

In [30]:
# number of commas
def count_commas(text):
    counter = 0
    tokens = word_tokenize(text.lower())
    
    for word in tokens:
        if word == ',':
            counter += 1
    return counter

In [31]:
test_comma = "I don't have a comma."
test_comma2 = "I have a comma, I also have a comma, and I have a comma again."

assert not count_commas(test_comma)
assert count_commas(test_comma2) == 2

#### The number of multi-character punctuation tokens

In [32]:
# number of multi-character punctuation tokens

def count_multi_punc(text, include_dots=True):
    counter = 0
    
    if include_dots:
        pattern = r"(\!{2,}|\?{2,}|\.{3,})"
    else:
        pattern = r"(\!{2,}|\?{2,})"
    
    compiled = re.compile(pattern)
    
    for match in compiled.finditer(text.lower()):
        if match:
            counter += 1
            
    return counter

In [33]:
test_punc = "I have a lot of exclamation marks!!!!!"
test_punc2 = "I don't have any."
test_punc3 = "Why am I so curious???? And I am also angry!!!!"
test_punc4 = "I am concerned...."

assert count_multi_punc(test_punc) == 1
assert not count_multi_punc(test_punc2)
assert count_multi_punc(test_punc3) == 2
assert count_multi_punc(test_punc4) == 1
assert not count_multi_punc(test_punc4, include_dots=False)

#### The number of positive words

In [34]:
###### Attribution for positive words dictionary: https://gist.github.com/mkulakowski2/4289437
positive_words = []
with open("data/helper_dicts/positive-words.txt") as f:
    for i in range(35):
        f.readline()

    for i in range(35, 2042):
        positive_words.append(f.readline().strip("\n"))

In [123]:
def get_avg_pos_words(text):
    tokens = word_tokenize(text)
    pos_count = 0

    for word in tokens:
        if word in positive_words:
            pos_count += 1

    return pos_count / len(tokens)

# avg pos = count / num. toknes

In [132]:
test_pos = "I am very happy!"
assert get_avg_pos_words(test_pos) == 1 / 5

#### The number of negative words

In [37]:
###### Attribution for negative words dictionary: https://gist.github.com/mkulakowski2/4289441

negative_words = []
with open("data/helper_dicts/negative-words.txt") as f:
    for i in range(35):
        f.readline()

    for i in range(35, 2042):
        negative_words.append(f.readline().strip("\n"))

In [130]:
def get_avg_neg_words(text):
    tokens = word_tokenize(text)
    neg_count = 0

    for word in tokens:
        if word in negative_words:
            neg_count += 1

    return neg_count / len(tokens)

In [131]:
test_neg = "I am annoyed."
assert get_avg_neg_words(test_neg) == 1 / 4

#### Slang sentiment

In [42]:
###### This sentiment dictionary for slang words are from SlangSD, made by Liang Wu (Student, ASU), Fred Morstatter (Student, ASU), Huan Liu (Professor, ASU).
###### Link for SlangSD: http://liangwu.me/slangsd/
slang_df = pd.read_csv("data/helper_dicts/SlangSD.txt", delimiter="\t", names=["sentiment"])

In [133]:
def get_avg_slang_sent(text):
    slang_sent = 0
    split = text.split(" ")
    tokens = [token.strip("".join(PUNCTUATION_LIST)) for token in split]

    for word in tokens:
        if word in slang_df.index and word not in stopwords.words("english"):
            slang_sent += slang_df.loc[word]["sentiment"]
    
    return slang_sent / len(tokens) ## avg vs just raw sum

In [134]:
slang_test = "you are kawaii!!!"
slang_test2 = "smh lmao tbh."
print(get_avg_slang_sent(slang_test))
print(get_avg_slang_sent(slang_test2))

0.6666666666666666
0.3333333333333333


#### The number of common nouns, proper nouns, adverbs, and wh- words

In [135]:
def count_tags(text):
    '''Count the number of common nouns, proper nouns, adverb, and wh- words'''
    
    words = word_tokenize(text)
    tagged_words = nltk.pos_tag(words)
    
    common_noun_count = 0
    proper_noun_count = 0
    adv_count = 0
    wh_count = 0
    
    for word, pos in tagged_words:
        if pos == "NN" or pos == "NNS":
            common_noun_count += 1
        elif pos == "NNPS" or pos == "NNP":
            proper_noun_count += 1
        elif pos == "RB" or pos == "RBR" or pos == "RBS":
            adv_count += 1
        elif pos == "WP" or pos == "WDT" or pos == "WRB":
            wh_count += 1
    return common_noun_count, proper_noun_count, adv_count, wh_count

In [136]:
text1 = "Michael likes to read books"
text2 = "Michael occasionally visits Finland"
text3 = "When will you go to Finland"
assert count_tags(text1) == (1, 1, 0, 0)
assert count_tags(text2) == (0, 2, 1, 0)
assert count_tags(text3) == (0,1, 0, 1)

#### The number of capitalized words

In [137]:
def count_cap_words(text):
    '''Count the amount of capitalized words in a text'''
    cap_words = 0
    words = word_tokenize(text)
    for word in words:
        if word.isupper():
            cap_words = cap_words +1
        else:
            cap_words = cap_words
    return cap_words

In [138]:
test_cap_words1 = "It IS ALL GOOD"
test_cap_words2 = "WOW I HAVE NEVER HEARD OF THAT"
assert count_cap_words(test_cap_words1) == 3
assert count_cap_words(test_cap_words2) == 7

#### Number of slang acronyms

In [139]:
#14 number of slang acronyms

def num_slang_acronym(text):
    '''Count the amount of slang acronyms in a text'''
    return len(re.findall(r"\b({})\b".format("|".join(SLANG)), text.lower()))

In [140]:
assert(num_slang_acronym("smh he wants to drive.") == 1)
assert(num_slang_acronym("smh lmao.") == 2)
assert(num_slang_acronym(" can't believe the vaccine is here") == 0)
assert(num_slang_acronym("can't believe. the vaccine is here") == 0)
assert(num_slang_acronym("smh. lmao.") == 2)
print("PASSED")

PASSED


#### Average length of sentences, in tokens

In [141]:
#15 avg length of sentences, in tokens

def avg_len_sent(text):
    '''Calculates the average length of sentences, in tokens'''
    token_count = len(text.split())
    sent_count = text.count(". ") + 1
    if sent_count != 0:
        return token_count / sent_count
    else:
        return 0

In [142]:
assert(avg_len_sent("smh he wants to drive.") == 5)
assert(avg_len_sent("he wants to drive. She wants to sit") == 4)
assert(avg_len_sent("he. wants. to. sit.") == 1)
assert(avg_len_sent(" he. wants. to. sit.") == 1)
print("PASSED")

PASSED


#### Average length of tokens, excluding punctuation tokens, in characters

In [143]:
#16 avg length of tokens, excluding punctuation-only tokens, in characters
#r"\b\w+?(?=\/)\b", 
tokenizer = RegexpTokenizer(r'\w+')

def avg_len_tokens(text):
    '''Calculates the average length of tokens, excluding punctuation, in characters'''
    token_with_no_punc = tokenizer.tokenize(text.lower())
    if len(token_with_no_punc) != 0:
        return len("".join(token_with_no_punc)) / len(token_with_no_punc) 
    else:
        return 0


In [144]:
assert(avg_len_tokens("smh he wants to drive.") == 3.4)
assert(avg_len_tokens("I can't believe the vaccine is here.") == 3.5)
assert(avg_len_tokens("I") == 1)
assert(avg_len_tokens(" I can't believe the vaccine is here.") == 3.5)
assert(avg_len_tokens("I can't believe. The vaccine is here.") == 3.5)
print("PASSED")


PASSED


#### Number of sentences

In [145]:
#17 number of sentences
def num_of_sent(text):
    '''Counts the number of sentences'''
    return text.count(". ") + 1

In [146]:
assert(num_of_sent("he wants to drive. She wants to sit") == 2)
assert(num_of_sent("he wants to drive.... She wants to sit") == 2)
assert(num_of_sent("he. wants. to. sit") == 4)
assert(num_of_sent(" he. wants. to. sit") == 4)
print("PASSED")

PASSED


In [147]:
## Test if all functions are working

count_past_tense(test_all)
count_first_person_pro(test_all)
count_second_person_pro(test_all)
count_third_person_pro(test_all)
count_coord_conj(test_all)
count_future_tense(test_all)
count_commas(test_all)
count_multi_punc(test_all)

0

### Apply all feature functions

In [148]:
df = pd.read_csv("data/processed/clean_data_7760_v2.csv")

df["past_tense_count"] = df["full_text"].apply(count_past_tense)
df["first_preson_pron_count"] = df["full_text"].apply(count_first_person_pro)
df["second_person_pron_count"] = df["full_text"].apply(count_second_person_pro)
df["third_person_pron_count"] = df["full_text"].apply(count_third_person_pro)
df["coord_conj_count"] = df["full_text"].apply(count_coord_conj)
df["future_tense_count"] = df["full_text"].apply(count_future_tense)
df["commas_count"] = df["full_text"].apply(count_commas)
df["multi_punc_count"] = df["full_text"].apply(count_multi_punc)
df["tags_count"] = df["full_text"].apply(count_tags)
df["cap_words_count"] = df["full_text"].apply(count_cap_words)
df["slang_acronym_count"] = df["full_text"].apply(num_slang_acronym)
df["sentence_count"] = df["full_text"].apply(num_of_sent)
df["avg_pos_words"] = df["full_text"].apply(get_avg_pos_words)
df["avg_neg_words"] = df["full_text"].apply(get_avg_neg_words)
df["avg_slang_sent"] = df["full_text"].apply(get_avg_slang_sent)
df["avg_len_sent"] = df["full_text"].apply(avg_len_sent)
df["avg_len_tokens"] = df["full_text"].apply(avg_len_tokens)

In [149]:
df.head()

,created_at,id,full_text,entities,is_quote_status,retweet_count,favorite_count,possibly_sensitive,keyword,location,...,multi_punc_count,tags_count,cap_words_count,slang_acronym_count,sentence_count,avg_pos_words,avg_neg_words,avg_slang_sent,avg_len_sent,avg_len_tokens
0,Mon Dec 21 17:31:03 +0000 2020,1.341074e+18,Today we add an important chapter to a Europea...,"['COVID19', 'EUvaccinationdays']",False,1557.0,5840.0,False,Covid19 vaccine,NaN,...,0,"(9, 3, 0, 0)",2,0,2,0.086957,0.0,-0.025641,20.000000,4.850000
1,Tue Dec 22 20:20:16 +0000 2020,1.341479e+18,Sedgwick County has received its first allocat...,['COVID19'],False,9.0,50.0,False,Covid19 vaccine,"Sedgwick County, Kansas",...,0,"(8, 6, 0, 1)",2,0,2,0.000000,0.0,0.000000,19.000000,5.421053
2,Wed Dec 23 00:32:16 +0000 2020,1.341542e+18,Now is not the time to let up. Vaccines are he...,['COVID19'],False,0.0,0.0,False,Covid19 vaccine,Southern California,...,0,"(13, 2, 4, 0)",1,0,2,0.020000,0.0,0.023256,21.500000,4.720930
3,Tue Dec 22 16:34:35 +0000 2020,1.341422e+18,I received my #COVID19 vaccination today. It w...,['COVID19'],False,139.0,1008.0,False,Covid19 vaccine,NaN,...,0,"(10, 1, 2, 0)",3,0,3,0.083333,0.0,0.000000,14.333333,4.627907
4,Sat Dec 19 18:58:07 +0000 2020,1.340371e+18,. Gen. Gus Perna on the distribution of the Mo...,['COVID19'],False,325.0,1784.0,False,Covid19 vaccine,"The Pentagon, Washington, D.C.",...,0,"(2, 5, 0, 0)",1,0,4,0.000000,0.0,0.000000,3.750000,4.642857


### Preprocessing

#### Checking location 

In [150]:
df.sentiment.unique()

array(['pos', 'neu', 'neg'], dtype=object)

In [151]:
df.country.unique()

array([nan, 'United States', 'Canada', 'Nigeria', 'United Kingdom',
       'United Arab Emirates', 'Australia', 'China', 'Kenya', 'Malaysia',
       'India', 'Ireland', 'Saudi Arabia', 'Belgium', 'South Africa',
       'Switzerland', 'Philippines', 'New Zealand', 'Netherlands',
       'Czech Republic', 'Malta', 'Spain', 'Singapore', 'French',
       'Costa Rica', 'Sweden', 'Mexico', 'Italy', 'Iran', 'Brazil',
       'Thailand', 'Jamaica', 'Pakistan', 'Finland', 'Marshall Islands',
       'Germany', 'South Korea', 'Sri Lanka', 'France', 'Israel',
       'Biafra', 'Portugal', 'Zimbabwe', 'Lebanon', 'Greece', 'Estonia',
       'The Gambia', 'Uganda', 'Turkey', 'Romania', 'Libya', 'Morocco',
       'Kuwait ', 'Norway', 'Venezuela', 'Qatar', 'England', 'Bulgaria',
       'Croatia', 'Bangladesh', 'Poland', 'Kuwait', 'Estonie',
       'Kurdistan', 'Mauritius', 'Czechia', 'Russia', 'Maldives', 'Japan',
       'Colombo', 'Gaia', 'Indonesia', 'Austria', 'Oman', 'Brasil',
       'Tunisia', 'Argen

In [56]:
df['city'] = df['city'].replace(['Washington'], [''])

In [57]:
df['country']= df['country'].replace(['USA', ' USA', 'USa', 'UK', 'Uited Kingdom','French', 'UAE'], 
                ['United States', 'United States', 'United States', 'United Kingdom', 'United Kingdom','France', 'United Arab Emirates'])

In [58]:
df['province']= df['province'].replace(['Qatar', 'Canada', 'TX United States'], ['','', 'TX'])

In [65]:
def tidy_province(text):
    if text == "D.C.":
        text = text.replace("D.C.", "DC")
    elif len(text) >= 3:
        text = text.replace("^([A-Za-z ]+)", "")
    return text

In [68]:
df.sentiment.value_counts()

neu    5329
neg    1390
pos    1041
Name: sentiment, dtype: int64

#### Exclude html referenced characters, punctuations, and digits

In [154]:
import string
def preprocessing(text):
    text = text.lower()
    exclude = string.punctuation + string.digits #exclude punctuations and digits
    for i in exclude:
        text = text.replace(i, '')
    return text

In [70]:
text1 = "I received my #COVID19 vaccine today &lt;"
text2 = "Sedgwick County has received its first allocation of COVID19 vaccine. #vaccine"
assert preprocessing(text1) == "i received my covid vaccine today "
assert preprocessing(text2) == "sedgwick county has received its first allocation of covid vaccine vaccine"

In [155]:
df["processed_text"] = df["full_text"].apply(preprocessing)

#### Lowercase and lemmatization

In [71]:
def lemmatize(text):
    """Lemmatize tweets by WordNetLemmatizer"""
    lemma_list = []
    lemmatizer = WordNetLemmatizer() 
    text = text.lower()
    words = word_tokenize(text)

    for word in words:
        lemma = lemmatizer.lemmatize(word, "n")
        if lemma == word:
            lemma = lemmatizer.lemmatize(word, "v")
        lemma_list.append(lemma)
  
    return ' '.join(lemma_list)

In [72]:
text1 = "They are being chased by hunters. We should be watching out."
text2 = "Have you seen flying dogs? No, I haven't."
assert lemmatize(text1) == "they be be chase by hunter . we should be watch out ."
assert lemmatize(text2) == "have you see fly dog ? no , i have n't ."

In [156]:
df["processed_text"] = df["processed_text"].apply(lemmatize)

#### Convert emoji into text representation

In [102]:
def convert_emoji_to_text(tweet):
    tokens = tweet.split()
    for i, token in enumerate(tokens):
        if token in UNICODE_EMOJI: 
            emo_desc = demoji.findall(token)[token]
            new_rep = "_".join(emo_desc.split(":")[0].split())
            tokens[i] = new_rep
    return " ".join(tokens)

In [103]:
assert convert_emoji_to_text("This plan is pretty 👌🏻 to me. We should 🤒") == 'This plan is pretty OK_hand to me. We should face_with_thermometer'
assert convert_emoji_to_text("I don't know how to write the function") == "I don't know how to write the function"

In [157]:
df["processed_text"] = df["processed_text"].apply(convert_emoji_to_text)

In [160]:
df.to_csv("data/processed/processed_data_7760.csv")